In [1]:
import os

In [2]:
%pwd

'e:\\NLP-Text-Summarization\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'e:\\NLP-Text-Summarization'

In [21]:
from pathlib import Path
from dataclasses import dataclass

@dataclass(frozen=True)
class Data_transformationCofig:
    root_dir  : Path
    data_path : Path
    tokiner_name : Path

In [26]:
from src.NLP_project.utils.common import create_directories,read_yaml
from src.NLP_project.constants import *

In [27]:
class Configuration:
    def __init__(self,config=config_path_file,params=params_path_file):
        self.config=read_yaml(config)
        self.params=read_yaml(params)

        create_directories([self.config.Artifacts])

    def get_data_transformation(self)->Data_transformationCofig:
        transformation=self.config.Data_transformation

        create_directories([transformation.root_dir])

        transformation=Data_transformationCofig(
            root_dir=transformation.root_dir,
            data_path=transformation.data_path,
            tokiner_name=transformation.tokiner_name)
        return transformation

    

In [18]:
import os

from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk
from src.NLP_project import logger

In [32]:
class Data_transformation:
    def __init__(self,config):
        self.config=config
        self.tokenizer=AutoTokenizer.from_pretrained(self.config.tokiner_name)
  
    def convert_examples_to_features(self,example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )
        
        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length = 128, truncation = True )
            
        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    
    
    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched = True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))




In [34]:
try:
    data_trans_config=Configuration()

    data_config=data_trans_config.get_data_transformation()

    transformations=Data_transformation(data_config)

    transformations.convert()
except Exception as e:
    raise e



[2023-09-05 13:51:47,727 - INFO - yaml file: config\config.yaml loaded successfully]
[2023-09-05 13:51:47,732 - INFO - yaml file: params.yaml loaded successfully]
[2023-09-05 13:51:47,734 - INFO - created directory at: artifacts]
[2023-09-05 13:51:47,736 - INFO - created directory at: artifacts/data_transformation]


c:\Users\HOME\anaconda3\envs\TEXTS\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\HOME\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Map:   0%|          | 0/14732 [00:00<?, ? examples/s]c:\Users\HOME\anaconda3\envs\TEXTS\lib\site-packages\transformers\tokenization_utils_ba